In [1]:
%maven org.dflib:dflib:2.0.0-M5
%maven org.dflib:dflib-csv:2.0.0-M5
%maven org.dflib:dflib-echarts:2.0.0-M5

In [2]:
import org.dflib.DataFrame;
import org.dflib.Series;
import org.dflib.ByteSource;
import org.dflib.csv.Csv;

import org.dflib.echarts.ECharts;
import org.dflib.echarts.EChartHtml;
import org.dflib.echarts.SeriesOpts;

import static org.dflib.Exp.*;
//import static org.dflib.jjava.runtime.Display.*;


In [3]:
static void render(EChartHtml chart) {
    String html = chart.getExternalScript()
            + chart.getContainer()
            + chart.getScript();

    // Do not leave the returned id as the last expression
    display(html, "text/html");
}


In [4]:
static DataFrame loadStooqDailyWithVolume(String symbol) {
    String url = "https://stooq.com/q/d/l/?s=" + symbol + "&i=d";
    ByteSource src = ByteSource.ofUrl(url);

    return Csv.load(src)
            .cols("Date", "Close", "Volume")
            .select(
                $str("Date"),
                $double("Close"),
                $double("Volume")
            )
            .sort($str("Date").asc());
}


In [5]:
static Series<Double> sma(Series<Double> s, int window) {
    int n = s.size();
    double[] out = new double[n];

    double sum = 0.0;

    for (int i = 0; i < n; i++) {
        Double vObj = s.get(i);
        double v = (vObj != null) ? vObj : Double.NaN;

        if (Double.isNaN(v)) {
            out[i] = Double.NaN;
            continue;
        }

        sum += v;

        if (i >= window) {
            Double oldObj = s.get(i - window);
            double oldV = (oldObj != null) ? oldObj : Double.NaN;
            if (!Double.isNaN(oldV)) sum -= oldV;
        }

        out[i] = (i >= window - 1) ? (sum / window) : Double.NaN;
    }

    return Series.ofDouble(out);
}


In [6]:
static String[] columnAsStringArray(DataFrame df, String col) {
    Series<?> s = df.getColumn(col);
    int n = df.height();
    String[] out = new String[n];
    for (int i = 0; i < n; i++) {
        Object v = s.get(i);
        out[i] = (v == null) ? null : v.toString();
    }
    return out;
}

static double[] columnAsDoubleArray(DataFrame df, String col) {
    Series<?> s = df.getColumn(col);
    int n = df.height();
    double[] out = new double[n];
    for (int i = 0; i < n; i++) {
        Object v = s.get(i);
        if (v == null) out[i] = Double.NaN;
        else if (v instanceof Number num) out[i] = num.doubleValue();
        else out[i] = Double.parseDouble(v.toString());
    }
    return out;
}


In [16]:
DataFrame aaplRaw = loadStooqDailyWithVolume("aapl.us");
DataFrame spxRaw  = loadStooqDailyWithVolume("^spx");

// Rename Close -> AAPL_Close / SPX_Close using cols(resultNames...).select(sourceExprs...)
DataFrame aapl = aaplRaw
        .cols("Date", "AAPL_Close")
        .select(
                $str("Date"),
                $double("Close")
        );

DataFrame spx = spxRaw
        .cols("Date", "SPX_Close")
        .select(
                $str("Date"),
                $double("Close")
        );

// sanity check
aapl.head(3)


ColumnDataFrame [{Date:1984-09-07,AAPL_Close:0.0996047},{Date:1984-09-10,AAPL_Close:0.0990135},{Date:1984-09-11,AAPL_Close:0.100827}]

In [17]:
String[] dates = columnAsStringArray(aapl, "Date");
double[] close = columnAsDoubleArray(aapl, "AAPL_Close");

Series<String> dateS  = Series.of(dates);
Series<Double> closeS = Series.ofDouble(close);

Series<Double> ma50  = sma(closeS, 50);
Series<Double> ma200 = sma(closeS, 200);

DataFrame aaplWithMA = DataFrame
        .byColumn("Date", "AAPL_Close", "MA50", "MA200")
        .of(dateS, closeS, ma50, ma200);

aaplWithMA.tail(3)


ColumnDataFrame [{Date:2025-12-30,AAPL_Close:273.08,MA50:272.46980000000036,MA200:231.46099999999944},{Date:2025-12-31,AAPL_Close:271.86,MA50:272.6622000000004,MA200:231.75029999999944},{Date:2026-01-02,AAPL_Close:271.01,MA50:272.8270000000004,MA200:232.04189999999943}]

In [18]:
DataFrame df = aaplWithMA
        .join(spx)
        .on("Date")
        .select();

df.tail(3)



ColumnDataFrame [{Date:2025-12-30,AAPL_Close:273.08,MA50:272.46980000000036,MA200:231.46099999999944,Date_:2025-12-30,SPX_Close:6896.24},{Date:2025-12-31,AAPL_Close:271.86,MA50:272.6622000000004,MA200:231.75029999999944,Date_:2025-12-31,SPX_Close:6845.5},{Date:2026-01-02,AAPL_Close:271.01,MA50:272.8270000000004,MA200:232.04189999999943,Date_:2026-01-02,SPX_Close:6858.47}]

In [20]:
EChartHtml aaplChart = ECharts.chart()
        .title("AAPL Close with SMA(50) and SMA(200)")
        .xAxis("Date")
        .series(SeriesOpts.ofLine(), "AAPL_Close", "MA50", "MA200")
        .plot(df);

render(aaplChart);


<script type='text/javascript' src='https://cdn.jsdelivr.net/npm/echarts@6.0.0/dist/echarts.min.js'></script><div id='dfl_ech_3678' class='dfl_ech' style='width: 600px;height:400px;'></div><script type='text/javascript'>var chart_dfl_ech_3678 = echarts.init(
        document.getElementById('dfl_ech_3678'),
        null,
   );
    var option_dfl_ech_3678 = {
        title: {
            text: 'AAPL Close with SMA(50) and SMA(200)',
        },
        dataset: {
            source: [
                ['L0','1984-09-07','1984-09-10','1984-09-11','1984-09-12','1984-09-13','1984-09-14','1984-09-17','1984-09-18','1984-09-19','1984-09-20','1984-09-21','1984-09-24','1984-09-25','1984-09-26','1984-09-27','1984-09-28','1984-10-01','1984-10-02','1984-10-03','1984-10-04','1984-10-05','1984-10-08','1984-10-09','1984-10-10','1984-10-11','1984-10-12','1984-10-15','1984-10-16','1984-10-17','1984-10-18','1984-10-19','1984-10-22','1984-10-23','1984-10-24','1984-10-25','1984-10-26','1984-10-29','1984-10-30','1984-10-31','1984-11-01','1984-11-02','1984-11-05','1984-11-06','1984-11-07','1984-11-08','1984-11-09','1984-11-12','1984-11-13','1984-11-14','1984-11-15','1984-11-16','1984-11-19','1984-11-20','1984-11-21','1984-11-23','1984-11-26','1984-11-27','1984-11-28','1984-11-29','1984-11-30','1984-12-03','1984-12-04','1984-12-05','1984-12-06','1984-12-07','1984-12-10','1984-12-11','1984-12-12','1984-12-13','1984-12-14','1984-12-17','1984-12-18','1984-12-19','1984-12-20','1984-12-21','1984-12-24','1984-12-26','1984-12-27','1984-12-28','1984-12-31','1985-01-02','1985-01-03','1985-01-04','1985-01-07','1985-01-08','1985-01-09','1985-01-10','1985-01-11','1985-01-14','1985-01-15','1985-01-16','1985-01-17','1985-01-18','1985-01-21','1985-01-22','1985-01-23','1985-01-24','1985-01-25','1985-01-28','1985-01-29','1985-01-30','1985-01-31','1985-02-01','1985-02-04','1985-02-05','1985-02-06','1985-02-07','1985-02-08','1985-02-11','1985-02-12','1985-02-13','1985-02-14','1985-02-15','1985-02-19','1985-02-20','1985-02-21','1985-02-22','1985-02-25','1985-02-26','1985-02-27','1985-02-28','1985-03-01','1985-03-04','1985-03-05','1985-03-06','1985-03-07','1985-03-08','1985-03-11','1985-03-12','1985-03-13','1985-03-14','1985-03-15','1985-03-18','1985-03-19','1985-03-20','1985-03-21','1985-03-22','1985-03-25','1985-03-26','1985-03-27','1985-03-28','1985-03-29','1985-04-01','1985-04-02','1985-04-03','1985-04-04','1985-04-08','1985-04-09','1985-04-10','1985-04-11','1985-04-12','1985-04-15','1985-04-16','1985-04-17','1985-04-18','1985-04-19','1985-04-22','1985-04-23','1985-04-24','1985-04-25','1985-04-26','1985-04-29','1985-04-30','1985-05-01','1985-05-02','1985-05-03','1985-05-06','1985-05-07','1985-05-08','1985-05-09','1985-05-10','1985-05-13','1985-05-14','1985-05-15','1985-05-16','1985-05-17','1985-05-20','1985-05-21','1985-05-22','1985-05-23','1985-05-24','1985-05-28','1985-05-29','1985-05-30','1985-05-31','1985-06-03','1985-06-04','1985-06-05','1985-06-06','1985-06-07','1985-06-10','1985-06-11','1985-06-12','1985-06-13','1985-06-14','1985-06-17','1985-06-18','1985-06-19','1985-06-20','1985-06-21','1985-06-24','1985-06-25','1985-06-26','1985-06-27','1985-06-28','1985-07-01','1985-07-02','1985-07-03','1985-07-05','1985-07-08','1985-07-09','1985-07-10','1985-07-11','1985-07-12','1985-07-15','1985-07-16','1985-07-17','1985-07-18','1985-07-19','1985-07-22','1985-07-23','1985-07-24','1985-07-25','1985-07-26','1985-07-29','1985-07-30','1985-07-31','1985-08-01','1985-08-02','1985-08-05','1985-08-06','1985-08-07','1985-08-08','1985-08-09','1985-08-12','1985-08-13','1985-08-14','1985-08-15','1985-08-16','1985-08-19','1985-08-20','1985-08-21','1985-08-22','1985-08-23','1985-08-26','1985-08-27','1985-08-28','1985-08-29','1985-08-30','1985-09-03','1985-09-04','1985-09-05','1985-09-06','1985-09-09','1985-09-10','1985-09-11','1985-09-12','1985-09-13','1985-09-16','1985-09-17','1985-09-18','1985-09-19','1985-09-20','1985-09-23','1985-09-24','1985-09-25','1985-09-26','1985-09-30','1985-1

In [21]:
double[] a = columnAsDoubleArray(df, "AAPL_Close");
double[] s = columnAsDoubleArray(df, "SPX_Close");
String[] d = columnAsStringArray(df, "Date");

int n = df.height();

// find first non-NaN values
double a0 = Double.NaN, s0 = Double.NaN;
for (int i = 0; i < n && (Double.isNaN(a0) || Double.isNaN(s0)); i++) {
    if (Double.isNaN(a0) && !Double.isNaN(a[i])) a0 = a[i];
    if (Double.isNaN(s0) && !Double.isNaN(s[i])) s0 = s[i];
}

double[] a100 = new double[n];
double[] s100 = new double[n];
for (int i = 0; i < n; i++) {
    a100[i] = (Double.isNaN(a[i]) || Double.isNaN(a0)) ? Double.NaN : (a[i] / a0) * 100.0;
    s100[i] = (Double.isNaN(s[i]) || Double.isNaN(s0)) ? Double.NaN : (s[i] / s0) * 100.0;
}

DataFrame rebased = DataFrame
        .byColumn("Date", "AAPL_100", "SPX_100")
        .of(Series.of(d), Series.ofDouble(a100), Series.ofDouble(s100));

EChartHtml rebasedChart = ECharts.chart()
        .title("AAPL vs S&P 500 (rebased to 100)")
        .xAxis("Date")
        .series(SeriesOpts.ofLine(), "AAPL_100", "SPX_100")
        .plot(rebased);

render(rebasedChart);


<script type='text/javascript' src='https://cdn.jsdelivr.net/npm/echarts@6.0.0/dist/echarts.min.js'></script><div id='dfl_ech_6023' class='dfl_ech' style='width: 600px;height:400px;'></div><script type='text/javascript'>var chart_dfl_ech_6023 = echarts.init(
        document.getElementById('dfl_ech_6023'),
        null,
   );
    var option_dfl_ech_6023 = {
        title: {
            text: 'AAPL vs S&amp;P 500 (rebased to 100)',
        },
        dataset: {
            source: [
                ['L0','1984-09-07','1984-09-10','1984-09-11','1984-09-12','1984-09-13','1984-09-14','1984-09-17','1984-09-18','1984-09-19','1984-09-20','1984-09-21','1984-09-24','1984-09-25','1984-09-26','1984-09-27','1984-09-28','1984-10-01','1984-10-02','1984-10-03','1984-10-04','1984-10-05','1984-10-08','1984-10-09','1984-10-10','1984-10-11','1984-10-12','1984-10-15','1984-10-16','1984-10-17','1984-10-18','1984-10-19','1984-10-22','1984-10-23','1984-10-24','1984-10-25','1984-10-26','1984-10-29','1984-10-30','1984-10-31','1984-11-01','1984-11-02','1984-11-05','1984-11-06','1984-11-07','1984-11-08','1984-11-09','1984-11-12','1984-11-13','1984-11-14','1984-11-15','1984-11-16','1984-11-19','1984-11-20','1984-11-21','1984-11-23','1984-11-26','1984-11-27','1984-11-28','1984-11-29','1984-11-30','1984-12-03','1984-12-04','1984-12-05','1984-12-06','1984-12-07','1984-12-10','1984-12-11','1984-12-12','1984-12-13','1984-12-14','1984-12-17','1984-12-18','1984-12-19','1984-12-20','1984-12-21','1984-12-24','1984-12-26','1984-12-27','1984-12-28','1984-12-31','1985-01-02','1985-01-03','1985-01-04','1985-01-07','1985-01-08','1985-01-09','1985-01-10','1985-01-11','1985-01-14','1985-01-15','1985-01-16','1985-01-17','1985-01-18','1985-01-21','1985-01-22','1985-01-23','1985-01-24','1985-01-25','1985-01-28','1985-01-29','1985-01-30','1985-01-31','1985-02-01','1985-02-04','1985-02-05','1985-02-06','1985-02-07','1985-02-08','1985-02-11','1985-02-12','1985-02-13','1985-02-14','1985-02-15','1985-02-19','1985-02-20','1985-02-21','1985-02-22','1985-02-25','1985-02-26','1985-02-27','1985-02-28','1985-03-01','1985-03-04','1985-03-05','1985-03-06','1985-03-07','1985-03-08','1985-03-11','1985-03-12','1985-03-13','1985-03-14','1985-03-15','1985-03-18','1985-03-19','1985-03-20','1985-03-21','1985-03-22','1985-03-25','1985-03-26','1985-03-27','1985-03-28','1985-03-29','1985-04-01','1985-04-02','1985-04-03','1985-04-04','1985-04-08','1985-04-09','1985-04-10','1985-04-11','1985-04-12','1985-04-15','1985-04-16','1985-04-17','1985-04-18','1985-04-19','1985-04-22','1985-04-23','1985-04-24','1985-04-25','1985-04-26','1985-04-29','1985-04-30','1985-05-01','1985-05-02','1985-05-03','1985-05-06','1985-05-07','1985-05-08','1985-05-09','1985-05-10','1985-05-13','1985-05-14','1985-05-15','1985-05-16','1985-05-17','1985-05-20','1985-05-21','1985-05-22','1985-05-23','1985-05-24','1985-05-28','1985-05-29','1985-05-30','1985-05-31','1985-06-03','1985-06-04','1985-06-05','1985-06-06','1985-06-07','1985-06-10','1985-06-11','1985-06-12','1985-06-13','1985-06-14','1985-06-17','1985-06-18','1985-06-19','1985-06-20','1985-06-21','1985-06-24','1985-06-25','1985-06-26','1985-06-27','1985-06-28','1985-07-01','1985-07-02','1985-07-03','1985-07-05','1985-07-08','1985-07-09','1985-07-10','1985-07-11','1985-07-12','1985-07-15','1985-07-16','1985-07-17','1985-07-18','1985-07-19','1985-07-22','1985-07-23','1985-07-24','1985-07-25','1985-07-26','1985-07-29','1985-07-30','1985-07-31','1985-08-01','1985-08-02','1985-08-05','1985-08-06','1985-08-07','1985-08-08','1985-08-09','1985-08-12','1985-08-13','1985-08-14','1985-08-15','1985-08-16','1985-08-19','1985-08-20','1985-08-21','1985-08-22','1985-08-23','1985-08-26','1985-08-27','1985-08-28','1985-08-29','1985-08-30','1985-09-03','1985-09-04','1985-09-05','1985-09-06','1985-09-09','1985-09-10','1985-09-11','1985-09-12','1985-09-13','1985-09-16','1985-09-17','1985-09-18','1985-09-19','1985-09-20','1985-09-23','1985-09-24','1985-09-25','1985-09-26','1985-09-30','1985-1

In [22]:
static DataFrame filterByDateRange(
        DataFrame df,
        String startDate,
        String endDate) {

    return df
        .rows(r -> {
            String d = r.get("Date").toString();
            return d.compareTo(startDate) >= 0
                && d.compareTo(endDate)   <= 0;
        })
        .select();
}

In [23]:
render(
    ECharts.chart()
        .title("AAPL Close with SMA(50) and SMA(200)")
        .xAxis("Date")
        .series(SeriesOpts.ofLine(), "AAPL_Close", "MA50", "MA200")
        .plot(filterByDateRange(df, "2022-01-01", "2026-01-03"))
);

<script type='text/javascript' src='https://cdn.jsdelivr.net/npm/echarts@6.0.0/dist/echarts.min.js'></script><div id='dfl_ech_3524' class='dfl_ech' style='width: 600px;height:400px;'></div><script type='text/javascript'>var chart_dfl_ech_3524 = echarts.init(
        document.getElementById('dfl_ech_3524'),
        null,
   );
    var option_dfl_ech_3524 = {
        title: {
            text: 'AAPL Close with SMA(50) and SMA(200)',
        },
        dataset: {
            source: [
                ['L0','2022-01-03','2022-01-04','2022-01-05','2022-01-06','2022-01-07','2022-01-10','2022-01-11','2022-01-12','2022-01-13','2022-01-14','2022-01-18','2022-01-19','2022-01-20','2022-01-21','2022-01-24','2022-01-25','2022-01-26','2022-01-27','2022-01-28','2022-01-31','2022-02-01','2022-02-02','2022-02-03','2022-02-04','2022-02-07','2022-02-08','2022-02-09','2022-02-10','2022-02-11','2022-02-14','2022-02-15','2022-02-16','2022-02-17','2022-02-18','2022-02-22','2022-02-23','2022-02-24','2022-02-25','2022-02-28','2022-03-01','2022-03-02','2022-03-03','2022-03-04','2022-03-07','2022-03-08','2022-03-09','2022-03-10','2022-03-11','2022-03-14','2022-03-15','2022-03-16','2022-03-17','2022-03-18','2022-03-21','2022-03-22','2022-03-23','2022-03-24','2022-03-25','2022-03-28','2022-03-29','2022-03-30','2022-03-31','2022-04-01','2022-04-04','2022-04-05','2022-04-06','2022-04-07','2022-04-08','2022-04-11','2022-04-12','2022-04-13','2022-04-14','2022-04-18','2022-04-19','2022-04-20','2022-04-21','2022-04-22','2022-04-25','2022-04-26','2022-04-27','2022-04-28','2022-04-29','2022-05-02','2022-05-03','2022-05-04','2022-05-05','2022-05-06','2022-05-09','2022-05-10','2022-05-11','2022-05-12','2022-05-13','2022-05-16','2022-05-17','2022-05-18','2022-05-19','2022-05-20','2022-05-23','2022-05-24','2022-05-25','2022-05-26','2022-05-27','2022-05-31','2022-06-01','2022-06-02','2022-06-03','2022-06-06','2022-06-07','2022-06-08','2022-06-09','2022-06-10','2022-06-13','2022-06-14','2022-06-15','2022-06-16','2022-06-17','2022-06-21','2022-06-22','2022-06-23','2022-06-24','2022-06-27','2022-06-28','2022-06-29','2022-06-30','2022-07-01','2022-07-05','2022-07-06','2022-07-07','2022-07-08','2022-07-11','2022-07-12','2022-07-13','2022-07-14','2022-07-15','2022-07-18','2022-07-19','2022-07-20','2022-07-21','2022-07-22','2022-07-25','2022-07-26','2022-07-27','2022-07-28','2022-07-29','2022-08-01','2022-08-02','2022-08-03','2022-08-04','2022-08-05','2022-08-08','2022-08-09','2022-08-10','2022-08-11','2022-08-12','2022-08-15','2022-08-16','2022-08-17','2022-08-18','2022-08-19','2022-08-22','2022-08-23','2022-08-24','2022-08-25','2022-08-26','2022-08-29','2022-08-30','2022-08-31','2022-09-01','2022-09-02','2022-09-06','2022-09-07','2022-09-08','2022-09-09','2022-09-12','2022-09-13','2022-09-14','2022-09-15','2022-09-16','2022-09-19','2022-09-20','2022-09-21','2022-09-22','2022-09-23','2022-09-26','2022-09-27','2022-09-28','2022-09-29','2022-09-30','2022-10-03','2022-10-04','2022-10-05','2022-10-06','2022-10-07','2022-10-10','2022-10-11','2022-10-12','2022-10-13','2022-10-14','2022-10-17','2022-10-18','2022-10-19','2022-10-20','2022-10-21','2022-10-24','2022-10-25','2022-10-26','2022-10-27','2022-10-28','2022-10-31','2022-11-01','2022-11-02','2022-11-03','2022-11-04','2022-11-07','2022-11-08','2022-11-09','2022-11-10','2022-11-11','2022-11-14','2022-11-15','2022-11-16','2022-11-17','2022-11-18','2022-11-21','2022-11-22','2022-11-23','2022-11-25','2022-11-28','2022-11-29','2022-11-30','2022-12-01','2022-12-02','2022-12-05','2022-12-06','2022-12-07','2022-12-08','2022-12-09','2022-12-12','2022-12-13','2022-12-14','2022-12-15','2022-12-16','2022-12-19','2022-12-20','2022-12-21','2022-12-22','2022-12-23','2022-12-27','2022-12-28','2022-12-29','2022-12-30','2023-01-03','2023-01-04','2023-01-05','2023-01-06','2023-01-09','2023-01-10','2023-01-11','2023-01-12','2023-01-13','2023-01-17','2023-01-18','2023-01-19','2023-01-20','2023-01-23','2023-01-24','2023-01-25','2023-01-26','2023-0

In [14]:
DataFrame aaplRaw = loadStooqDailyWithVolume("aapl.us");

DataFrame aapl = aaplRaw
        .cols("Date", "AAPL_Close", "Volume")
        .select(
            $str("Date"),
            $double("Close"),
            $double("Volume")
        );

// extract arrays (reuse your helpers)
String[] dates   = columnAsStringArray(aapl, "Date");
double[] close   = columnAsDoubleArray(aapl, "AAPL_Close");
double[] volume  = columnAsDoubleArray(aapl, "Volume");

Series<String> dateS  = Series.of(dates);
Series<Double> closeS = Series.ofDouble(close);
Series<Double> volS   = Series.ofDouble(volume);

Series<Double> ma50  = sma(closeS, 50);
Series<Double> ma200 = sma(closeS, 200);

DataFrame aaplWithMA = DataFrame
        .byColumn("Date", "AAPL_Close", "MA50", "MA200", "Volume")
        .of(dateS, closeS, ma50, ma200, volS);

aaplWithMA.tail(3)


ColumnDataFrame [{Date:2025-12-30,AAPL_Close:273.08,MA50:272.46980000000036,MA200:231.46099999999944,Volume:2.2139617E7},{Date:2025-12-31,AAPL_Close:271.86,MA50:272.6622000000004,MA200:231.75029999999944,Volume:2.7293639E7},{Date:2026-01-02,AAPL_Close:271.01,MA50:272.8270000000004,MA200:232.04189999999943,Volume:3.7838054E7}]

In [24]:
import org.dflib.DataFrame;
import org.dflib.echarts.*;

EChartHtml chart = ECharts.chart()
    .title("AAPL — Price (MA50/MA200) & Volume")
    .xAxis("Date")

    // Y0: price (left)
    .yAxis(YAxis.ofValue().name("Price").left())

    // Y1: volume (right)
    .yAxis(YAxis.ofValue().name("Volume").right())

    // Price series -> yAxisIndex(0)
    .series(SeriesOpts.ofLine().yAxisIndex(0), "AAPL_Close")
    .series(SeriesOpts.ofLine().yAxisIndex(0), "MA50")
    .series(SeriesOpts.ofLine().yAxisIndex(0), "MA200")

    // Volume series -> yAxisIndex(1)
    .series(SeriesOpts.ofBar().yAxisIndex(1), "Volume")

    .plot(aapl);   // df must contain: Date, AAPL_Close, MA50, MA200, AAPL_Volume


RuntimeException: java.lang.IllegalArgumentException, Value 'MA50' is not present in the Index